In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

/Users/Cemal/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
properties = pd.read_csv('./input/properties_2016.csv')
train = pd.read_csv("./input/train_2016_v2.csv")
for c in properties.columns:
    properties[c]=properties[c].fillna(-1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))

/Users/Cemal/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_df = train.merge(properties, how='left', on='parcelid')
x_train = train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
x_test = properties.drop(['parcelid'], axis=1)
# shape        
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

Shape train: (90275, 57)
Shape test: (2985217, 57)


In [4]:
# drop out ouliers
train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.4 ]
x_train=train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)

print('After removing outliers:')     
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

After removing outliers:
Shape train: (88431, 57)
Shape test: (2985217, 57)


In [5]:
# xgboost params
xgb_params = {
    'eta': 0.06,
    'max_depth': 5,
    'subsample': 0.75,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

In [6]:
# cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   nfold=5,
                   num_boost_round=200,
                   early_stopping_rounds=50,
                   verbose_eval=10, 
                   show_stdv=False
                  )
num_boost_rounds = len(cv_result)
print(num_boost_rounds)

[0]	train-mae:0.052841	test-mae:0.0528574
[10]	train-mae:0.0524052	test-mae:0.0525496
[20]	train-mae:0.0521776	test-mae:0.0524384
[30]	train-mae:0.052019	test-mae:0.05239
[40]	train-mae:0.0518944	test-mae:0.0523628
[50]	train-mae:0.0517914	test-mae:0.0523488
[60]	train-mae:0.0517	test-mae:0.0523374
[70]	train-mae:0.05161	test-mae:0.0523266
[80]	train-mae:0.0515376	test-mae:0.0523248
[90]	train-mae:0.0514578	test-mae:0.0523208
[100]	train-mae:0.0513908	test-mae:0.052319
[110]	train-mae:0.051315	test-mae:0.052323
[120]	train-mae:0.05124	test-mae:0.0523248
[130]	train-mae:0.051174	test-mae:0.052321
88


In [7]:
# train model
watchlist = [(dtrain, 'train'), (dtest, 'valid')]
model = xgb.train(dict(xgb_params, silent=1), dtrain, verbose_eval=10)
#model = xgb.train(dict(xgb_params, silent=1), 1000, dtrain, watchlist)#, num_boost_round=88)
pred = model.predict(dtest)
y_pred=[]

for i,predict in enumerate(pred):
    y_pred.append(str(round(predict,4)))
y_pred=np.array(y_pred)

output = pd.DataFrame({'ParcelId': properties['parcelid'].astype(np.int32),
        '201610': y_pred, '201611': y_pred, '201612': y_pred,
        '201710': y_pred, '201711': y_pred, '201712': y_pred})

In [8]:
# set col 'ParceID' to first col
cols = output.columns.tolist()
cols = cols[-1:] + cols[:-1]

In [9]:
output = output[cols]
from datetime import datetime
output.to_csv('xgb_without_outliers.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)